In [ ]:
from bokeh.io import show, output_notebook
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper
)
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure

from bokeh.sampledata.us_counties import data as counties
import pandas

output_notebook()
palette.reverse()

# Choose a state to plot
state = "TX"
###### Read our data
# Estimated Use of Water in the United States
field = 'DO-TOTAL '  # Domestic, total use (withdrawals + deliveries), in Mgal/d
df = pandas.read_excel('../data/Humans and Hydrology at High Latitudes: Water Use Information/usco2005.xls')
df[df.STATE == state][field]
county_rates = list(df[field])

counties = {
    code: county for code, county in counties.items() if county["state"] == state.lower()
}

county_xs = [county["lons"] for county in counties.values()]
county_ys = [county["lats"] for county in counties.values()]
county_names = [county['name'] for county in counties.values()]
color_mapper = LogColorMapper(palette=palette)
source = ColumnDataSource(data=dict(
    x=county_xs,
    y=county_ys,
    name=county_names,
    rate=county_rates,
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title="Estimated Use of Water in households, 2005", tools=TOOLS,
    x_axis_location=None, y_axis_location=None
)
p.grid.grid_line_color = None

p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("County", "@name"),
    ("Use [Mgal/d]", "@rate"),
    ("(Long, Lat)", "($x, $y)"),
]

show(p)